In [1]:
# Importing Relevant Dependencies
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as st
import datetime as dt
import numpy as np


cdcPath = "../../../COVID-19_Case_Surveillance_Public_Use_Data.csv"

In [ ]:
# Create data frame from CSV file variable(path) using pandas .read_csv() method
cdcDf = pd.read_csv(cdcPath, low_memory=False)

# Print out first 10 rows for inspection
cdcDf.head(3)

In [ ]:
# Dropping unnecessary columns
cutCdcDf = cdcDf.drop(['pos_spec_dt', 'onset_dt','icu_yn','death_yn'], axis=1)

cutCdcDf.head()

In [ ]:
# Checking values in Race and Ethnicity to identify possible invalid/missing values 
cdcDf['Race and ethnicity (combined)'].value_counts()

In [ ]:
# Dropping missing or unknown data using conditional formatting
ethUnkDf = cutCdcDf[(cutCdcDf['Race and ethnicity (combined)'] != 'Missing') &
                    (cutCdcDf['Race and ethnicity (combined)'] != 'Unknown') ]
ethUnkDf.head()

In [ ]:
# Re-Checking values in Race and Ethnicity for conditional formatting accuracy
ethUnkDf['Race and ethnicity (combined)'].value_counts()

In [ ]:
# Checking values in the Age Groups
ethUnkDf['age_group'].value_counts()

In [ ]:
# Dropping unknown values from the age groups
unkAgeDf = ethUnkDf[ethUnkDf['age_group'] != 'Unknown']

# Checking for accuracy
unkAgeDf.head()

In [ ]:
# Checking hospitilization values
unkAgeDf['hosp_yn'].value_counts()

In [ ]:
# Dropping missing and unknown values from hospitilization
misHospDf = unkAgeDf[(unkAgeDf['hosp_yn'] != 'Missing') &
                     (unkAgeDf['hosp_yn'] != 'Unknown')]

misHospDf.head(10)

In [ ]:
# Checking values for medical conditions
misHospDf['medcond_yn'].value_counts()

In [ ]:
# Dropping missing and unknown values from conditions
misMedDf = misHospDf[(misHospDf['medcond_yn'] != 'Missing') &
                     (misHospDf['medcond_yn'] != 'Unknown')]

misMedDf.head(10)

In [ ]:
# Checking values in the sex column
misMedDf['sex'].value_counts()

In [ ]:
# Removing missing and unknown values from the sex column
misSexDf = misMedDf[(misMedDf['sex'] != 'Missing') &
                    (misMedDf['sex'] != 'Unknown') &
                    (misMedDf['sex'] != 'Other') ]

misSexDf.head()

In [ ]:
# Checking values in the status column
misSexDf['current_status'].value_counts()

In [ ]:
# Removing non-laboratory confirmed cases
cleanCdcDf = misSexDf[misSexDf['current_status'] != 'Probable Case']
cleanCdcDf.head(10)

In [ ]:
# Renaming Column Headers for Visualization and Analyis Purposes
rnCdcDf = cleanCdcDf.rename(columns={"cdc_report_dt": "CDC Report Date",
                                     "current_status": "Covid Status",
                                     "sex": "Gender",
                                     "age_group": "Age Group",
                                     "Race and ethnicity (combined)": "Race/Ethnicity",
                                     "hosp_yn": "Hospitalized",
                                     "medcond_yn": "Pre-Existing Condition"})
# Sort values by CDC Report Date
sorted_CdcDf = rnCdcDf.sort_values("CDC Report Date", ascending=True)

# Reset index and save to new variable
resetCdcDf = sorted_CdcDf.reset_index(drop=True)

resetCdcDf.head()

In [ ]:
# Convert Date column to correct "date" type
resetCdcDf['CDC Report Date'] = pd.to_datetime(resetCdcDf['CDC Report Date'], format= '%Y/%m/%d')

# Create new column for months of the date for grouping purposes and change to "numerical" type
resetCdcDf['CDC Report Month (#)'] = resetCdcDf['CDC Report Date'].dt.strftime('%m')
resetCdcDf['CDC Report Month (#)'] = pd.to_numeric(resetCdcDf['CDC Report Month (#)'])

# Create bins to aggregate the data into months and use pd.cut() method to bin the data into a new column
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November"]

resetCdcDf['CDC Report Month'] = pd.cut(resetCdcDf['CDC Report Month (#)'], bins, labels=months, include_lowest=False)
resetCdcDf.head()

In [ ]:
# Create variable for refence when calling aggregates by Month
monthCdcDf = resetCdcDf.groupby("CDC Report Month")

# Below is simply an example, you can perform any aggregates youd like 
count_df = monthCdcDf.sum()
count_df['Cases Per Day'] = ""
count_df['Cases Per Day'] = [ round(28/31, 2), round(294/28, 2), round(90843/31, 2),
                             round(332736/30, 2), round(435255/31, 2), round(884196/30, 2),
                             round(1000902/31, 2), round(1028376/31, 2), round(942156/30, 2),
                             round(1423530/31, 2), round(1377959/19, 2) ]
count_df

In [ ]:
## Create a line graph based on the group series to track total cases over the period of the year 
# Set x-axis and tick locations
x_axis = np.arange(len(months))
tick_locations = [month for month in x_axis]

# Create line chart tracking the change in cases per day thorughout the year
avg_daily = count_df['Cases Per Day'] 
plt.plot(x_axis, avg_daily, color="red", marker="o", label="Cases per Day", linestyle="dashed")

# Set the labels, axes, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Average Cases per Day 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Day")

plt.tight_layout()
plt.show()
plt.savefig("../Images/CasesPerDay.png")

In [ ]:
## GENDER vs. HOSPITALIZATION

In [ ]:
# Data Frame Specific to the Analysis of Gender vs. Hospitalization Status 
patient_sex_df = resetCdcDf[["CDC Report Month", "Gender", "Hospitalized"]]
patient_sex_df.head()

In [ ]:
# The Data Frame below in the superimposition of the two line graphs following

In [ ]:
# Create a data frame grouped by Month and Gender
gender_month_df = resetCdcDf.groupby(["CDC Report Month", "Gender"])
gender_month_df.sum()
gender_avg = gender_month_df.sum()

# Pivot table for readability and plotting purposes
gender_avg_pivot = gender_avg.pivot_table('CDC Report Month (#)', index='CDC Report Month', columns='Gender')


## Create a line graph based on the Gender
male_avg, = plt.plot(months, gender_avg_pivot['Male'], color="blue", label="Male Avg")
female_avg, = plt.plot(months, gender_avg_pivot['Female'], color="red", label="Female Avg")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Average Cases per Month by Gender 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[male_avg, female_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/GenderCasesPerMonth.png")

gender_avg_pivot
gender_avg

In [ ]:
#Make Data Frame only including columns where Hospitalized is True
hos_gender_df = patient_sex_df[patient_sex_df['Hospitalized'] == "Yes"]
hos_gender_df.head()

# Create a data frame grouped by Month and Pre-Existing Condition
gender_hos_month_df = hos_gender_df.groupby(["CDC Report Month", "Gender"])
gender_hos_avg = gender_hos_month_df.count()
gender_hos_avg

# Pivot table for readability and plotting purposes
gender_hos_avg_pivot = gender_hos_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Gender')
gender_hos_avg_pivot

In [ ]:
## Create a line graph based on the Gender (Hospitalization = Yes)
hos_gen_fem_avg, = plt.plot(months, gender_hos_avg_pivot['Female'], color="red", label="Female")
hos_gen_male_avg, = plt.plot(months, gender_hos_avg_pivot['Male'], color="blue", label="Male")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Hospitalization Cases per Month by Gender 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[hos_gen_fem_avg, hos_gen_male_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/HospitalizationsGenderMonth.png")

In [ ]:
#Make Data Frame only including columns where Hospitalized is False
ok_gender_df = patient_sex_df[patient_sex_df['Hospitalized'] == "No"]
ok_gender_df.head()

# Create a data frame grouped by Month and Gender
gender_ok_month_df = ok_gender_df.groupby(["CDC Report Month", "Gender"])
gender_ok_avg = gender_ok_month_df.count()
gender_ok_avg

# Pivot table for readability and plotting purposes
gender_ok_avg_pivot = gender_ok_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Gender')
gender_ok_avg_pivot

In [ ]:
## Create a line graph based on the Gender (Hospitalization = No)
ok_gen_fem_avg, = plt.plot(months, gender_ok_avg_pivot['Female'], color="red", label="Female")
ok_gen_male_avg, = plt.plot(months, gender_ok_avg_pivot['Male'], color="blue", label="Male")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Non-Hospitalization Cases per Month by Gender 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[ok_gen_fem_avg, ok_gen_male_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/NonHospGenderMonth.png")

In [ ]:
## AGE GROUP vs. HOSPITALIZATION

In [ ]:
# Data Frame Specific to the Analysis of Age Group vs. Hospitalization Status
# Removed uneeded column
patient_age_df = resetCdcDf[["CDC Report Month", "Hospitalized", "Age Group"]]
patient_age_df.head()

In [ ]:
# Make a copy of the original cleaned/reset data 
# This is necessary bc Python/Pandas does not like using any iterative methods on a dataframe that is a slice of the original 

resetCdcDf1 = resetCdcDf.copy()
resetCdcDf1.head()

# Replace Age Groups with Age Divisions
resetCdcDf1['Age Group'] = resetCdcDf1['Age Group'].replace({'50 - 59 Years':'Older', '60 - 69 Years':'Older', '70 - 79 Years':'Older', '80+ Years':'Older'})
resetCdcDf1['Age Group'] = resetCdcDf1['Age Group'].replace({'0 - 9 Years':'Younger', '10 - 19 Years':'Younger', '20 - 29 Years':'Younger', '30 - 39 Years':'Younger', '40 - 49 Years':'Younger'})

# Rename this Column to 'Age Division'
renamedCdcDf1 = resetCdcDf1.rename(columns={'Age Group':'Age Division'})

renamedCdcDf1.head(20)

In [ ]:
# Now, Make a Data Frame Specific to the Analysis of Age Division vs. Hospitalization Status
patient_age_df = renamedCdcDf1[["CDC Report Month", "Hospitalized", "Age Division"]]
patient_age_df.head(10)

In [ ]:
# For Line Chart displaying Hospitalizations by Age Division
#Make Data Frame only including columns where Hospitalized is True
hos_age_df = patient_age_df[patient_age_df['Hospitalized'] == "Yes"]
hos_age_df.head()

# Create a data frame grouped by Month and Pre-Existing Condition
age_hos_month_df = hos_age_df.groupby(["CDC Report Month", "Age Division"])
age_hos_avg = age_hos_month_df.count()
age_hos_avg

# Pivot table for readability and plotting purposes
age_hos_avg_pivot = age_hos_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Age Division')
age_hos_avg_pivot

In [ ]:
## Create a line graph based on the Age Division (Hospitalization = Yes)
hos_old_avg, = plt.plot(months, age_hos_avg_pivot['Older'], color="red", label="50+ Years")
hos_young_avg, = plt.plot(months, age_hos_avg_pivot['Younger'], color="blue", label="Less than 50 Years")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Hospitalization Cases per Month by Age Division 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[hos_old_avg, hos_young_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/HospitalizationAgeMonth.png")

In [ ]:
# For Line Chart displaying Non-Hospitalizations by Age Division
#Make Data Frame only including columns where Hospitalized is True
ok_age_df = patient_age_df[patient_age_df['Hospitalized'] == "No"]
ok_age_df.head()

# Create a data frame grouped by Month and Pre-Existing Condition
age_ok_month_df = ok_age_df.groupby(["CDC Report Month", "Age Division"])
age_ok_avg = age_ok_month_df.count()
age_ok_avg

# Pivot table for readability and plotting purposes
age_ok_avg_pivot = age_ok_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Age Division')
age_ok_avg_pivot

In [ ]:
## Create a line graph based on the Age Division (Hospitalization = Yes)
ok_old_avg, = plt.plot(months, age_ok_avg_pivot['Older'], color="red", label="50+ Years")
ok_young_avg, = plt.plot(months, age_ok_avg_pivot['Younger'], color="blue", label="Less than 50 Years")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Non-Hospitalization Cases per Month by Age Division 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[hos_old_avg, hos_young_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/NonHosAgeMonth.png")

In [ ]:
# Make empty column to divide age groups into two 'Divisions': Younger & Older
# Iterate through rows of the data frame and append divisors into the new, empty column 
#resetCdcDf['Age Division'] = ""
#resetCdcDf.head(10)


## Trying to build for loop to fill new empty column with appended values
## (Tried)Implement for loop to loop through rows of data frame and append 'Age Division' values

# Set initial varaible that contains the default string you want to append divisor to the new column
#patient_var = "Older"

#for patient in resetCdcDf:
#    if resetCdcDf['Age Group'] == '50 - 59 Years':
#        resetCdcDf['Age Division'].append(patient_var)
#    elif resetCdcDf['Age Group'] == '60 - 69 Years':
#       resetCdcDf['Age Division'].append(patient_var)
#    elif resetCdcDf['Age Group'] == '70 - 79 Years':
#        resetCdcDf['Age Division'].append(patient_var)
#    elif resetCdcDf['Age Group'] == '80+ Years':
#        presetCdcDf['Age Division'].append(patient_var)  
#    else:
#        patient_var = "Younger"
#        resetCdcDf['Age Division'].append(patient_var)



# Build function that takes original reset/cleaned data frame and makes a copy
# Using for loop and the index of the row, append correct Younger/Older divisor to that row    
        

    
#for i in resetCdcDf.index:
#    if (resetCdcDf.loc[i, 'Age Group'] == '50-59 Years'):
#            resetCdcDf.at[i, 'Age Division'] = 'Older'
#    elif (resetCdcDf.loc[i, 'Age Group'] == '60-69 Years'):
#            resetCdcDf.at[i, 'Age Division'] = 'Older'
#    elif (resetCdcDf.loc[i, 'Age Group'] == '70-79 Years'):
#            resetCdcDf.at[i, 'Age Division'] = 'Older'
#    elif (resetCdcDf.loc[i, 'Age Group'] == '80+ Years'):
#            resetCdcDf[i, 'Age Division'] = 'Older'
#    else:
#            resetCdcDf.at[i, 'Age Division'] = 'Younger'


#resetCdcDf.head(10)

In [ ]:
## PRE-EXISTING CONDITION vs. HOSPITALIZATION

In [ ]:
# Data Frame Specific to the Analysis of Pre-Exisiting Condition vs. Hospitalization Status
patient_premed_df = resetCdcDf[["CDC Report Month", "Hospitalized", "Pre-Existing Condition"]]
patient_premed_df.head()

In [ ]:
# Make Data Frame only including columns where Hospitalized is True
hos_patient_premed_df = patient_premed_df[patient_premed_df['Hospitalized'] == "Yes"]
hos_patient_premed_df.head()

# Create a data frame grouped by Month and Pre-Existing Condition
prehos_month_df = hos_patient_premed_df.groupby(["CDC Report Month", "Pre-Existing Condition"])
prehos_month_df
prehos_avg = prehos_month_df.count()
prehos_avg

# Pivot table for readability and plotting purposes
prehos_avg_pivot = prehos_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Pre-Existing Condition')
prehos_avg_pivot

In [ ]:
## Create a line graph based on the Pre-Existing Medical Condition (Hospitalization = Yes)
hosexis_avg, = plt.plot(months, prehos_avg_pivot['Yes'], color="red", label="Existing")
hosnon_avg, = plt.plot(months, prehos_avg_pivot['No'], color="blue", label="Non-Existing")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Hospitalization per Month by Pre-Existing Medical Conditions 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[hosexis_avg, hosnon_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/HospitalizationsPreExisMonth.png")

In [ ]:
# Make Data Frame only including columns where Hospitalized is False
ok_patient_premed_df = patient_premed_df[patient_premed_df['Hospitalized'] == "No"]
ok_patient_premed_df.head()

# Create a data frame grouped by Month and Pre-Existing Condition
ok_month_df = ok_patient_premed_df.groupby(["CDC Report Month", "Pre-Existing Condition"])
ok_avg = ok_month_df.count()
ok_avg

# Pivot table for readability and plotting purposes
ok_avg_pivot = ok_avg.pivot_table('Hospitalized', index='CDC Report Month', columns='Pre-Existing Condition')
ok_avg_pivot

In [ ]:
## Create a line graph based on the Pre-Existing Medical Condition (For Non-Hospitalization)
okexis_avg, = plt.plot(months, ok_avg_pivot['Yes'], color="red", label="Existing")
oknon_avg, = plt.plot(months, ok_avg_pivot['No'], color="blue", label="Non-Existing")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Non-Hospitalization per Month by Pre-Existing Medical Conditions 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[okexis_avg, oknon_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/NonHosPreExisMonth.png")

In [ ]:
# Create a data frame grouped by Month and Pre-Existing Condition
pre_month_df = resetCdcDf.groupby(["CDC Report Month", "Pre-Existing Condition"])
pre_month_df.sum()
pre_avg = pre_month_df.sum()
pre_avg


# Pivot table for readability and plotting purposes
pre_avg_pivot = pre_avg.pivot_table('CDC Report Month (#)', index='CDC Report Month', columns='Pre-Existing Condition')
pre_avg_pivot

In [ ]:
## Create a line graph based on the Pre-Existing Medical Condition
## This displays the two charts above sumperimposed on eachother
exis_avg, = plt.plot(months, pre_avg_pivot['Yes'], color="red", label="Existing")
non_avg, = plt.plot(months, pre_avg_pivot['No'], color="blue", label="Non-Existing")

# Set the labels, axes, legend, and title
plt.xticks(tick_locations, months, rotation="vertical")
plt.title("US Average Cases per Month by Pre-Existing Medical Condition Status 2020")
plt.xlabel("Months")
plt.ylabel("Cases per Month")
plt.legend(handles=[exis_avg, non_avg], loc="best")

plt.tight_layout()
plt.show()
plt.savefig("../Images/CasesPreExisMonth.png")

pre_avg_pivot
pre_avg